# [LVV-T2730] - StarTracker Pointing and Tracking Test - Forward Azimuth Pattern

Collect data with the StarTracker following the azimuth pattern -270, -180, -90, 0, 90, 180, 270 deg.  
Nominal at four elevation angle 15, 45, 75, 86.5 deg.  
Minimum at the three angle: 15, 45, 86.5 deg.  

This test is forseen the first of four tests takes about one summer night (7 hours) in full version and a bit more than 5hours in the shortened version.  

Can we track for 10 min without moving the dome?

**Requirements:**
 - All the MT components should be enabled.
 - Need the `notebooks_vandv` module installed.

Please, see the [README] file for the requirements to run this notebook.  
The log messages printed in this notebook are stored in the EFD Script using the `-TTTTMMDD` format, where `TTTT` are the four last digits of the test case, `MM` is the 0-padded month, and `DD` is the 0-padded day of execution.

[README]: https://github.com/lsst-sitcom/notebooks_vandv/blob/develop/README.md
[LVV-T2730]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2730

## Setup

In [18]:
test_case = "LVV-T2730"
test_exec = "LVV-EXXXX"

az_grid = [-270, -180, -90, 0, 90, 180, 270]
# el_grid = [16, 45, 75, 86.5] # Nominal Elevation Angle - M1M3 goes to fault at 16 deg.
el_grid = [20, 45, 75, 86.5] # Nominal Elevation Angle
el_grid_min = [15, 45, 86.5] # Minimum Elevation Angle

mag_limit = 4
mag_range = 2
radius = 5.0

camera_sal_indexes = [1, 2]
exposure_times = [10., 5.]  # s
track_time = 30.  # 10 min
base_msg = f"{test_case} {test_exec}:"

### Prepare Notebook

Start with importing libraries:

In [19]:
%load_ext autoreload
%autoreload 2

import asyncio
import logging
import sys
import yaml

from astropy.time import Time

from lsst.ts import salobj
from lsst.ts.idl.enums.Script import ScriptState
from lsst.ts.observatory.control import RotType
from lsst.ts.observatory.control.maintel import MTCS, ComCam
from lsst.ts.observatory.control.generic_camera import GenericCamera

from lsst.sitcom import vandv

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


---
Print out the execution info for future reference.

In [20]:
exec_info = vandv.ExecutionInfo()
print(exec_info)


Executed by b1quint on 2022-11-08T19:29:28.492.
  Running in pillan06 at tucson



---
Use the `maintel/setup_mtcs.py` script in LOVE or the [LVV-T2344] to test case and notebook to setup all the main telescope components.  
This includes simulators as well as real hardware when available (this will depend on when the test is conducted at TTS or on level 3 or on the telescope):  

- pointing  
- mount ( with the CCW)  
- rotator  
- ready M1M3: raise mirror, turn on FB, clear forces. Note that if used at level 3, we need to have M1M3 LUT use mount telemetry  
- ready M2: turn on FB, clear forces. Note that if used at level 3, we need to have M2 LUT use mount telemetry  
- Get cam hex Ready: check config; make sure LUT is on and has valid inputs; make sure hex is at LUT position  
- Get M2 hex (simulator) Ready: check config; make sure LUT is on and has valid inputs; make sure hex is at LUT position  
- Finally, get the MTAOS CSC ready  

[LVV-T2344]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2344

---
Setup the logger, the Domain, and the remote for the telescope control.

In [21]:
logging.basicConfig(format="%(asctime)s %(name)s: %(message)s", level=logging.DEBUG)
log = logging.getLogger("setup")
log.level = logging.INFO

In [22]:
os.environ["LSST_DDS_HISTORYSYNC"] = "200"
domain = salobj.Domain()
print(f"My user_host is {domain.user_host!r}\n")

My user_host is 'b1quint@nb-b1quint'



Instanciate the MTCS.

In [23]:
mtcs = MTCS(domain=domain, log=log)
await asyncio.sleep(2.5)
await mtcs.start_task

[None, None, None, None, None, None, None, None, None, None]

Initialize the GenericCamera and ComCam.

In [24]:
camera_list = []

for index in camera_sal_indexes:

    gencam = GenericCamera(index=index)
    await asyncio.sleep(2.5)
    await gencam.start_task

    camera_list.append(gencam)
    
    break
    

comcam = ComCam()
await asyncio.sleep(2.5)
await comcam.start_task

camera_list.append(comcam)

Run the cell below to hide all the messages related to the DDS and have a cleaner notebook.

In [25]:
vandv.logger.add_filter_to_mtcs()

---
The `Script` CSC is used to record test checkpoints and allow to easy search in the EFD.

In [26]:
script = salobj.Controller("Script", index=vandv.get_index(test_case))
await asyncio.sleep(2.5) 
await script.start_task


  Using script index: -27301108



Default wait time in the steps.

In [27]:
STDWAIT = 20

In [28]:
await comcam.enable()

### Check CSC Status

In [29]:
await mtcs.assert_liveliness()

In [30]:
await mtcs.assert_all_enabled()

In [31]:
for cam in camera_list:
    await cam.assert_liveliness()
    await cam.assert_all_enabled()

### Other preparation

- LVV-T2713 (1.0) Establish TMA - StarTracker Axis Angle Reference

## Helper Functions

In [32]:
def generate_azel_sequence(az_seq, el_seq):
    """A generator that cicles through the input azimuth and elevation sequences
    forward and backwards.
    
    Parameters
    ----------
    az_seq : `list` [`float`]
        A sequence of azimuth values to cicle through
    el_seq : `list` [`float`]
        A sequence of elevation values to cicle through     
    Yields
    ------
    `list`
        Values from the sequence.
    Notes
    -----
    This generator is designed to generate sequence of values cicling through
    the input forward and backwards. It will also reverse the list when moving
    backwards.
    Use it as follows:
    >>> az_seq = [0, 180]
    >>> el_seq = [15, 45]
    >>> seq_gen = generate_azel_sequence(az_seq, el_seq)
    >>> next(seq_gen)
    [0, 15]
    >>> next(seq_gen)
    [0, 45]
    >>> next(seq_gen)
    [180, 45]
    >>> next(seq_gen)
    [180, 15]
    >>> next(seq_gen)
    [0, 15]
    """
    i = 1
    for az in az_seq:
        for el in el_seq[::i]:
            yield (az, el)
        i *= -1

In [33]:
async def take_images(cams, exp_times, n_exps, reason):
    """Takes images with multiple cameras at the same time.
    
    cams : `list`
        Contains camera instances.
    exp_times : `list` [`float`]
        The exposure time for each camera. 
    n_exps : `list` [`float`]
        The numer of exposures for each camera.
    reason : `str`
        Reason passed to the `take_object` command.
    """ 
    reason = reason.replace(" ", "_")
    
    # tasks = [cam.take_object(exptime, n=n_exp, reason=reason) 
    tasks = [asyncio.create_task(cam.take_object(exptime, n=n_exp, reason=reason))
             for (cam, exptime, n_exp) in zip(cams, exp_times, n_exps)]
    
    await asyncio.gather(*tasks)

## Data Acquisition

In [34]:
az_grid = [0]
el_grid = [71]

for az, el in generate_azel_sequence(az_grid, el_grid):

    ## All of the following steps should be included in the point_azel command
    ## - Point the Dome
    ## - Wait the Dome to arrive    
    ## - Point the TMA
    ## - Wait the TMA to arrive
    script.log.info(f"{base_msg} Data acquisition at az={az} and el={el} - Start")
    
    # Ignore error in the Dome for now
    mtcs.check.mtdome = False
    
    ## Enable dome following
    script.log.info(f"{base_msg} Dome following - Enable")
    await mtcs.enable_dome_following()    
    
    ## Point Az/El
    script.log.info(f"{base_msg} Point to az={az}, el={el} - Start")
    await mtcs.point_azel(az, el)
    script.log.info(f"{base_msg} Point to az={az}, el={el} - Done")
        
    ## Start tracking.
    script.log.info(f"{base_msg} Tracking - Start")
    await mtcs.rem.mtptg.cmd_startTracking.set_start()
    
    ## Disable dome following
    script.log.info(f"{base_msg} Dome following - Disable")
    await mtcs.disable_dome_following()
    
    n_images = [int(track_time / t_exp) for t_exp in exposure_times] 
    n_images_str = "/".join([str(i) for i in n_images])
    
    exp_start = Time.now()
    script.log.info(f"{base_msg} Take {n_images_str} image(s) while tracking - Start")
    await take_images(camera_list, exposure_times, n_images, base_msg[:-1]) 
    script.log.info(f"{base_msg} Take {n_images_str} image(s) while tracking - Done")

    exp_end = Time.now()
    delta = exp_end - exp_start
    delta.format = "sec"
    script.log.info(f"{base_msg} Images acquisition took {delta}.")
    
    ## Stop tracking
    script.log.info(f"{base_msg} Tracking - Stop")
    await mtcs.rem.mtptg.cmd_stopTracking.set_start()
    
    script.log.info(f"{base_msg} Data acquisition at az={az} and el={el} - Done")